##### Proyecto Final BEDU Python (3 y 4)
### Clasificador de Imágenes de RMN usandouna  red nueuronal convolucionada prediseñada.

`03/2021 - 07/2021`

> García Espinosa David Alexis 

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS

# Bibliotecas a utilizar.
import pandas as pd
import numpy as np 
import seaborn as sns
import os 
import matplotlib.pyplot as plt